# Antibody Design -  test

These are the rules to complete this test:

- In this test, you will be asked to perform some analyses on antibody sequences using python, and return the results as a jupyter notebook in a git repository.
- The test should not take more than a couple of hours to complete. You can use any tools on internet, but you can't receive any external help.
- You should describe the protocol you have used to obtain the results, whenever it involves anything outside the jupyter notebook itself (e.g. databases, web tools, etc)
- The results of the test will be discussed in the 2nd interview

The sequence of the antibody we will analyse is the following:



>VH
QVQLKESGPGLVAPSQSLSITCTVSGFPLTAYGVNWVRQPPGKGLEWLGMIWGDGNTDYNSALKSRLSISKDNSKSQVFLKMNSLQTDDTARYYCARDPYGSKPMDYWGQGTSVTVSS
>VL
DIVMSQSPSSLVVSVGEKVTMSCKSSQSLLYSSNQKNFLAWYQQKPGQSPKLLIYWASTRESGVPDRFTGSGSGTDFTLTISSVKAEDLAVYYCQQYFRYRTFGGGTKLEIKRA

__Q1__ Install the package anarci, and using it retrieve the following information:
- germlines
- CDR sequences
- SHMs

In [ ]:
# Creating csv for VH
!ANARCI -i QVQLKESGPGLVAPSQSLSITCTVSGFPLTAYGVNWVRQPPGKGLEWLGMIWGDGNTDYNSALKSRLSISKDNSKSQVFLKMNSLQTDDTARYYCARDPYGSKPMDYWGQGTSVTVSS --assign_germline --outfile ABDesign_case --csv --scheme imgt

# Creating csv for VL
!ANARCI -i DIVMSQSPSSLVVSVGEKVTMSCKSSQSLLYSSNQKNFLAWYQQKPGQSPKLLIYWASTRESGVPDRFTGSGSGTDFTLTISSVKAEDLAVYYCQQYFRYRTFGGGTKLEIKRA --assign_germline --outfile ABDesiign_case --csv --scheme imgt

In [ ]:
import pandas as pd

# Loading saved CSV
df_VH = pd.read_csv("ABDesign_case_H.csv")
df_VL = pd.read_csv("ABDesign_case_KL.csv")

In [ ]:
aligned_VH = df_VH.iloc[:, 13:]
aligned_VL = df_VL.iloc[:, 13:]

In [ ]:
# Providing overview of important information from the VH chain
VH_info = dict(
    v_gene=df_VH.v_gene.values[0],
    j_gene=df_VH.j_gene.values[0],
    CDR1="".join(aligned_VH.iloc[:, 26:38].values.tolist()[0]),
    CDR2="".join(aligned_VH.iloc[:, 55:65].values.tolist()[0]),
    CDR3="".join(aligned_VH.iloc[:, 104:117].values.tolist()[0]),
)
VH_info

In [ ]:
# Providing overview of important information from the VL chain
VL_info = dict(
    v_gene=df_VL.v_gene.values[0],
    j_gene=df_VL.j_gene.values[0],
    CDR1="".join(aligned_VL.iloc[:, 26:38].values.tolist()[0]),
    CDR2="".join(aligned_VL.iloc[:, 55:65].values.tolist()[0]),
    CDR3="".join(aligned_VL.iloc[:, 104:117].values.tolist()[0]),
)
VL_info

__Q2__ write a function that returns all the occurrences of the motifs NG, DG, NS, and DS, if they overlap at least partially with any of the CDRs

In [ ]:
def occurences(sequence):
    """
    A function that counts the occurences of overlapping motifs with any CDRs.

    Assuming sequence is IMGT-numbered
    """
    motifs = ["NG", "DG", "NS", "DS"]  # A list to look up motifs
    motif_counts = dict(NG=0, DG=0, NS=0, DS=0)  # dict where we count each motif
    for motif in motifs:
        if motif in sequence[25:39].replace("-", ""):
            motif_counts[motif] += 1
        if motif in sequence[54:66].replace("-", ""):
            motif_counts[motif] += 1
        if motif in sequence[103:118].replace("-", ""):
            motif_counts[motif] += 1
    return motif_counts

In [ ]:
# Checking occurences of motifs in light chain CDRs
print(f"Motifs in VL CDRs: {occurences(''.join(aligned_VL.values.tolist()[0]))}")
# Checking occurences of motifs in heavy chain CDRs
print(f"Motifs in VH CDRs: {occurences(''.join(aligned_VH.values.tolist()[0]))}")

Only one overlapping or partialle overlapping motif in CDR was found in the heavy chain

__Q3__ These sites are potential liabilities in the antibody. Using online tools, and your own intuition, describe which othese occurences might be problematic, which residues you would suggest mutating to eliminate such liabilities minimizing the risk of affecting the antibody binding affinity, and why

If the motifs are found at CDR regions it can affect binding. As CDRs are the ones that interact with antigen it is likely that a change in motif in these regions lead to poor affinity. Thus, I would suggest mutating residues in the framework regions to minimize risk affecting antibody binding affinity

__Q4__ Using the online tool ABodybuilder, build a 3d structure for this antibody. On the structure, using pymol or any python library of your choice (e.g. BioPDB) identify the minimal distance between any atom of any residue in the CDR3 of the heavy and light chains. You will need to report the residue in the CDR3 of the havy chain and the atom, the residue in the CDFR3 of the light chain and the atom

In [ ]:
from Bio.PDB.PDBParser import PDBParser

parser = PDBParser(PERMISSIVE=True)
structure_id = "ABDesign_case_rank1_imgt_scheme"
filename = "ABDesign_case_rank1_imgt_scheme.pdb"
model = parser.get_structure(structure_id, filename)[0]
chainH = model["H"]
chainL = model["L"]

In [ ]:
# Here I chose CA of residue 114 of the heavy chain and calculated its distance to the CA of residue 107 in the light chain
chainH[114]["CA"] - chainL[107]["CA"]

__Q5__ A library was constructed on the antibody by introducing variability in the CDR3 of the heavy chain, and the binding affinity was measured for each antibody in the library. The affinities are reported as -log10, so the higher values indicate the stronger binders. Can you provide the code to train an XGboost model on the train_data using the techniques you deem necessary to ensure the model robustness, and use it to predict the binding affinity of the antibody in the list _test_
Report the model performance in terms of RMSE, the predicted values for the test antibodies, and any comment on the model.

In [ ]:
import numpy as np
import random
import string

# Define the number of data points
n_samples = 1000

# Define possible amino acids
amino_acids = list("ARNDCEQGHILKMFPSTWYV")


# Generate random CDR3 sequences
def generate_CDR3(length):
    return "".join(random.choice(amino_acids) for _ in range(length))


CDR3_sequences = [generate_CDR3(np.random.randint(15, 20)) for _ in range(n_samples)]

# Generate random scores between 0 and 10
scores = np.random.uniform(6, 10, n_samples)

# Create dataset
train_data = list(zip(CDR3_sequences, scores))

test_data = [
    "KYWRDGSTWWELIYGYG",
    "RQSFKFCCTFE",
    "LLWRYFFRKFSCD",
    "STLTFSDNIGSYPCWD",
    "QRLQEHTAGRG",
]

In [ ]:
import sapiens


def get_sapiens_embedding(sequences):
    embs = []
    for seq in sequences:
        emb = sapiens.predict_sequence_embedding(seq, "H", layer=None)
        embs.append(emb)
    return np.array(embs)

In [ ]:
train_embeddings = get_sapiens_embedding(CDR3_sequences)
test_embeddings = get_sapiens_embedding(test_data)

I perform gridsearch and kfold cross validation in this case since the sequences are generated by random selection from the amino acids at random lengths. If these were real CDR3 sequences, I would expect more conservation at certain residues and would probably use something like a stratified k-fold CV to group similar sequences into the same fold. I want to predict how each of these mutations affect binding affinity and it will be harder for the model to learn if grouped randomly as in normal K-fold

Below I use the sklearn and a XGboost regressor to predict BA. I perform gridsearch over max depth of the tree, number of estimators and the embedding layers in the sapiens embeddings. I use Sapiens to model the sequences and obtain sequence embeddings. These are then scaled prior to PCA followed by fitting the model.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import make_scorer
import xgboost as xgb


# Defining RMSE loss function
def rmse(actual, predicted):
    predicted = np.array(predicted)
    actual = np.array(actual)
    difference = predicted - actual
    squared_difference = difference**2
    mean_squared_difference = squared_difference.mean()
    rmse = np.sqrt(mean_squared_difference)
    return rmse


# Wrap the scorer to be used with sklearn
scorer = make_scorer(rmse, greater_is_better=False)


# Defining a base estimator to extract a given layer
class sapiens_embeddings(BaseEstimator, TransformerMixin):
    def __init__(self, layer=None):
        self.layer = layer

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        emb = X[:, self.layer, :]
        return emb


# Defining XGBRegressor
xgb_model = xgb.XGBRegressor()

# Defining a pipleline starting with obtaining the given embeddings, then scaling, then PCA and are then fed to the model
pipe = Pipeline(
    steps=[
        ("embeddings", sapiens_embeddings()),
        ("scale", StandardScaler()),
        ("pca", PCA(n_components=0.95)),
        ("model", xgb_model),
    ]
)

# Define gridsearch using the pipeline and providing the parameters to search
clf = GridSearchCV(
    pipe,
    {
        "embeddings__layer": range(5),
        "model__max_depth": [2, 4, 6],
        "model__n_estimators": [100, 200, 400, 800],
    },
    verbose=1,
    scoring=scorer,
    n_jobs=-1,
    cv=5,
    refit=True,
)

# Fit the model
clf.fit(train_embeddings, scores)

In [ ]:
# Extracting best results
mean_score = clf.cv_results_["mean_test_score"][clf.best_index_]
print(f"Best parameters: {clf.best_params_}")
print(f"RMSE Mean CV score: {mean_score}")

Visualizing train predictions vs true values

In [ ]:
import matplotlib.pyplot as plt

train_y_pred = clf.predict(train_embeddings)
plt.scatter(train_y_pred, scores)
plt.xlim(5, 11)
plt.ylim(5, 11)
plt.xlabel("Predicted BA")
plt.ylabel("True BA")
plt.title("True vs Predicted BA's")
plt.show()

A linear trend is observed but predictions comes with high uncertainty.

Now I predict the test set

In [ ]:
clf.predict(test_embeddings)